## Processing pandas function walkthrough

In [10]:
import pandas as pd
import os
import json

In [12]:
# %load extract_skills
import spacy
from negspacy.negation import Negex  # noqa:F401
from typing import List, Dict


def extract_skills(description: str, inSkillsDict: Dict) -> List[str]:
    """Extracts skills from description string

    :param description: course description
    :type description: str
    :param inSkillsDict: dictionary of skills
    :type inSkillsDict: Dict
    :raises Exception: checks if description is not string
    :return: list of skills
    :rtype: List[str]
    """

    if type(description) != str:
        raise Exception("Input must be str")

    nlp = spacy.load("en_core_web_md")
    nlp.add_pipe("negex", after="ner", config={"ent_types": ["SKILL"]})
    ruler = nlp.add_pipe("entity_ruler", before="ner")

    patterns = []
    full_patterns = []

    for skill in inSkillsDict:
        patterns.extend(inSkillsDict[skill])

    words_list = [word.split() for word in patterns]

    skills_to_label_regardless_of_pos = ["agile"]

    formatted_skills = [skill.lower().split() for skill in skills_to_label_regardless_of_pos]

    for words in words_list:
        pattern = generate_pattern(words)
        full_patterns.append(pattern)

    final_patterns = [{"label": "SKILL", "pattern": pattern} for pattern in full_patterns]

    ruler.add_patterns(final_patterns)

    doc = nlp(description)

    words_to_remove = ["Unlike", "unlike"]

    filtered_sents = []

    for sent in doc.sents:
        if not any(word in sent.text for word in words_to_remove):
            filtered_sents.append(sent.text)

    filtered_doc = nlp(" ".join(filtered_sents))

    result = []

    for ent in filtered_doc.ents:
        if ent.label_ == "SKILL" and ent._.negex is False:
            result.append(ent.text)

    return result


def generate_pattern(words: List[str]) -> List[Dict]:
    """Generates spacy entity pattern

    :param words: List of individual skills
    :type words: List[str]
    :return: Spacy entity ruler patterns
    :rtype: List[Dict]
    """
    skills_to_label_regardless_of_pos = ["agile"]
    formatted_skills = [skill.lower().split() for skill in skills_to_label_regardless_of_pos]

    pattern = []

    for word in words:  
        if '-' in word:
            tokens = word.split('-')
            token_patterns = [{'LOWER': token.lower()} for token in tokens]
            token_patterns.insert(1, {'IS_PUNCT': True})
            pattern.extend(token_patterns)
        elif words in formatted_skills or len(words) > 1:
            pattern.append({"LOWER": word.lower()})
        else:
            pattern.append({"LOWER": word.lower(), "POS": {"IN": ["PROPN", "NOUN"]}})

    return pattern


### 1. Define test skills dictionary

This is a shortened example of a dictionary of skills and synonyms to extract data from

In [13]:
test_dict = {"JavaScript": ["Javascript"], "Angular": ["Angular"], "Ruby on Rails": ["Ruby on Rails"], "React": ["react", "react.js", "reactjs"], "Python": ["Python"], "Django": ["Django"], "Express": ["Express"], "Node.js": ["Node.js"], "SQL": ["SQL"], "Excel": ["Excel"], "PowerBI": ["PowerBI"], "Tableau": ["Tableau"], "HTML": ["HTML"], "Devops": ["Devops"], "CSS": ["CSS"], "Object-oriented programming": ["object-oriented programming"]}


In [14]:
test_str = 'JavaScript is no longer taught on this course, but Python is'

test_str2 = 'Unlike other courses that teach Python, we only teach JavaScript'

test_str3 = 'JavaScript is not taught on this course anymore, but Python is'

test_str4 = 'Students will create two websites (a 1-page website and a 5-page website) over the course of 12 weeks. Students will learn to code in HTML, CSS and Javascript. Students will experience the following: \r\n\r\n-Discover FTP (File Transfer Process) website servers\r\n-Develop link building skills\r\n-Learn the Bootstrap framework for responsive design\r\n-Learn how to font with Awesome icons\r\n-Learn how to use Photoshop\r\n-Learn how to implement contact forms. Unlike our Devops course, we do teach Python. We teach object-oriented programming'

test_str5 = "Become a software engineer in 13 weeks at our coding bootcamps in Manchester, Leeds, Newcastle, Birmingham and remotely.\r\n\r\nUnlike our Data Engineering bootcamp where you focus specifically on the \"back-end\" of software, or our DevOps Engineering bootcamp that deals specifically with software development and IT operations, our coding bootcamp focuses on building websites and mobile phone apps.\r\n\r\nThe application process takes 2-3 weeks and we would advise you to apply sooner rather than later to give yourself plenty of time to work through the preparation materials.\r\n\r\nApplicants living in England can apply for DfE funding to cover the entire cost of the course. Get in touch to find out if you qualify."

extracted_skills = extract_skills(test_str4, test_dict)

print(extracted_skills)


['HTML', 'CSS', 'Javascript', 'object-oriented programming']


### 2. Read json file into DataFrame

- The file contains a shortened raw data example

In [220]:
unprocessed_dataframe = pd.read_json("./example_data/raw_course_data.json")

### 3. Explode provider_courses column to create rows for each course

- `reset_index` creates unique id for each new row

In [221]:
exploded_courses = unprocessed_dataframe.explode('provider_courses').reset_index()

exploded_courses.head()

,index,provider_name,provider_locations,provider_tracks,provider_courses,meta
0,0,Codez Academy,[Wales],[Front End Developer],"{'course_name': 'Digital Roots Scheme', 'cours...",{'target_url': 'https://www.coursereport.com/s...
1,0,Codez Academy,[Wales],[Front End Developer],{'course_name': 'Foundation Course in Front-en...,{'target_url': 'https://www.coursereport.com/s...
2,1,School of Code,"[Birmingham, Liverpool, London, Manchester]",[Full Stack Developer],"{'course_name': 'School of Code Bootcamp', 'co...",{'target_url': 'https://www.coursereport.com/s...
3,2,Coders Lab,"[Amsterdam, Brussels, Bucharest, Edinburgh, Gl...","[Full Stack Developer, Data Science, Front End...","{'course_name': 'Automation Tester', 'course_s...",{'target_url': 'https://www.coursereport.com/s...
4,2,Coders Lab,"[Amsterdam, Brussels, Bucharest, Edinburgh, Gl...","[Full Stack Developer, Data Science, Front End...","{'course_name': 'Java Developer', 'course_skil...",{'target_url': 'https://www.coursereport.com/s...


### 4. Create new DataFrame based on provider_courses column keys

In [222]:
normalised_courses = pd.json_normalize(exploded_courses.provider_courses)

normalised_courses.head()

,course_name,course_skills,course_locations,course_description
0,Digital Roots Scheme,"[HTML, JavaScript, CSS]",Wales,"Students will learn three main languages: CSS,..."
1,Foundation Course in Front-end Development,"[HTML, CSS, JavaScript]",Wales,Students will create two websites (a 1-page we...
2,School of Code Bootcamp,"[C#, JavaScript, CSS, Design, Express.js, Fron...",Birmingham,Learn Full-Stack Web Development on our 16 wee...
3,Automation Tester,"[Java, Quality Assurance Testing]","Warsaw, Online, Kraków, Edinburgh, Glasgow, Li...",Do you think that if you have yet no clue abou...
4,Java Developer,"[Java, JavaScript, jQuery, MySQL, SQL, Git, Gi...","Warsaw, Vienna, Kraków, Online, Edinburgh, Gla...","During the course, you will learn Java for Int..."


## 5. Process course_descriptions




In [223]:

def consolidate_desc_into_skills(row):
    extracted_skills = extract_skills(str(row['course_description']), test_dict)
    
    existing_skills = row['course_skills']

    row['course_skills'] = list(set(existing_skills + extracted_skills))

    return row

normalised_courses.apply(lambda x: consolidate_desc_into_skills(x), axis=1)

df = pd.concat([normalised_courses['course_skills'], normalised_courses['course_description']], axis=1)

df.to_csv('./example_extraction.csv')

normalised_courses.drop('course_description', axis=1)


,course_name,course_skills,course_locations
0,Digital Roots Scheme,"[CSS, HTML, JavaScript, Javascript]",Wales
1,Foundation Course in Front-end Development,"[JavaScript, HTML, CSS, Javascript]",Wales
2,School of Code Bootcamp,"[Algorithms, Front End, CSS, User Experience D...",Birmingham
3,Automation Tester,"[Quality Assurance Testing, Java]","Warsaw, Online, Kraków, Edinburgh, Glasgow, Li..."
4,Java Developer,"[MySQL, Java, Git, GitHub, jQuery, SQL, HTML, ...","Warsaw, Vienna, Kraków, Online, Edinburgh, Gla..."
5,JavaScript Developer,"[CSS, MongoDB, jQuery, React.js, Node.js, HTML...","Warsaw, Jakarta, Madrid, Kraków, Online, Vienn..."
6,Manual Tester,"[CSS, MySQL, REST, Linux, HTML, Scrum, Agile]","Warsaw, Jakarta, Kraków, Online, Vienna, Edinb..."
7,Python Developer,"[Django, JavaScript, CSS, MySQL, Git, GitHub, ...","Warsaw, Jakarta, Kraków, Online, Edinburgh, Gl..."
8,ENTRY-LEVEL CLOUD ENGINEER,"[Front End, CSS, AngularJS, Scrum, MySQL, Git,...",Online
9,ENTRY-LEVEL SOFTWARE ENGINEER,"[Front End, Java, Cloud Computing, HTML, JavaS...",Online


### 6. Combine normalised provider_courses DataFrame with original DataFrame and drop unnecessary columns

- We're removing `provider_locations` as `course_locations` is also provided.

In [224]:
concat_dataframe_with_courses = pd.concat([exploded_courses, normalised_courses], axis=1).drop(['provider_courses', 'provider_locations', 'provider_tracks'], axis=1)

concat_dataframe_with_courses.head()

,index,provider_name,meta,course_name,course_skills,course_locations,course_description
0,0,Codez Academy,{'target_url': 'https://www.coursereport.com/s...,Digital Roots Scheme,"[CSS, HTML, JavaScript, Javascript]",Wales,"Students will learn three main languages: CSS,..."
1,0,Codez Academy,{'target_url': 'https://www.coursereport.com/s...,Foundation Course in Front-end Development,"[JavaScript, HTML, CSS, Javascript]",Wales,Students will create two websites (a 1-page we...
2,1,School of Code,{'target_url': 'https://www.coursereport.com/s...,School of Code Bootcamp,"[Algorithms, Front End, CSS, User Experience D...",Birmingham,Learn Full-Stack Web Development on our 16 wee...
3,2,Coders Lab,{'target_url': 'https://www.coursereport.com/s...,Automation Tester,"[Quality Assurance Testing, Java]","Warsaw, Online, Kraków, Edinburgh, Glasgow, Li...",Do you think that if you have yet no clue abou...
4,2,Coders Lab,{'target_url': 'https://www.coursereport.com/s...,Java Developer,"[MySQL, Java, Git, GitHub, jQuery, SQL, HTML, ...","Warsaw, Vienna, Kraków, Online, Edinburgh, Gla...","During the course, you will learn Java for Int..."


### 7. Explode course_skills column to add row per skill

- `reset_index` again for unique row ids and then drop unnecessary index columns.

In [225]:
exploded_skills = concat_dataframe_with_courses.explode('course_skills').reset_index().drop(['index', 'level_0'], axis=1)

exploded_skills.head()

,provider_name,meta,course_name,course_skills,course_locations,course_description
0,Codez Academy,{'target_url': 'https://www.coursereport.com/s...,Digital Roots Scheme,CSS,Wales,"Students will learn three main languages: CSS,..."
1,Codez Academy,{'target_url': 'https://www.coursereport.com/s...,Digital Roots Scheme,HTML,Wales,"Students will learn three main languages: CSS,..."
2,Codez Academy,{'target_url': 'https://www.coursereport.com/s...,Digital Roots Scheme,JavaScript,Wales,"Students will learn three main languages: CSS,..."
3,Codez Academy,{'target_url': 'https://www.coursereport.com/s...,Digital Roots Scheme,Javascript,Wales,"Students will learn three main languages: CSS,..."
4,Codez Academy,{'target_url': 'https://www.coursereport.com/s...,Foundation Course in Front-end Development,JavaScript,Wales,Students will create two websites (a 1-page we...


### 8. Create new DataFrame based on meta column keys

In [226]:
normalised_meta = pd.json_normalize(exploded_skills.meta)

normalised_meta.head()

,target_url,timestamp
0,https://www.coursereport.com/schools/codez-aca...,2023-05-25 14:12:01.312105
1,https://www.coursereport.com/schools/codez-aca...,2023-05-25 14:12:01.312105
2,https://www.coursereport.com/schools/codez-aca...,2023-05-25 14:12:01.312105
3,https://www.coursereport.com/schools/codez-aca...,2023-05-25 14:12:01.312105
4,https://www.coursereport.com/schools/codez-aca...,2023-05-25 14:12:01.312105


### 9. Combine normalised meta DataFrame with original DataFrame and drop unnecessary column

In [227]:
concat_dataframe_with_meta = pd.concat([exploded_skills, normalised_meta], axis=1).drop('meta', axis=1)

concat_dataframe_with_meta.head()

,provider_name,course_name,course_skills,course_locations,course_description,target_url,timestamp
0,Codez Academy,Digital Roots Scheme,CSS,Wales,"Students will learn three main languages: CSS,...",https://www.coursereport.com/schools/codez-aca...,2023-05-25 14:12:01.312105
1,Codez Academy,Digital Roots Scheme,HTML,Wales,"Students will learn three main languages: CSS,...",https://www.coursereport.com/schools/codez-aca...,2023-05-25 14:12:01.312105
2,Codez Academy,Digital Roots Scheme,JavaScript,Wales,"Students will learn three main languages: CSS,...",https://www.coursereport.com/schools/codez-aca...,2023-05-25 14:12:01.312105
3,Codez Academy,Digital Roots Scheme,Javascript,Wales,"Students will learn three main languages: CSS,...",https://www.coursereport.com/schools/codez-aca...,2023-05-25 14:12:01.312105
4,Codez Academy,Foundation Course in Front-end Development,JavaScript,Wales,Students will create two websites (a 1-page we...,https://www.coursereport.com/schools/codez-aca...,2023-05-25 14:12:01.312105


### 10. Map course_locations to list of strings and then explode for a row per location

- Note the block below reassigns itself so run the block above again before you rerun!

In [228]:
concat_dataframe_with_meta['course_locations'] = concat_dataframe_with_meta['course_locations'].map(lambda x: x.split(', '))

exploded_locations = concat_dataframe_with_meta.explode('course_locations')

course_locations = ['Online', 'Bath', 'Birmingham', 'Bristol', 'Buckinghamshire', 'Cambridge', 'Edinburgh', 'Glasgow', 'Leeds', 'Liverpool', 'London', 'Manchester', 'Sheffield', 'Wales', 'West Yorkshire']

exploded_locations_filtered = exploded_locations[exploded_locations['course_locations'].isin(course_locations)]

exploded_locations_filtered.loc[:,('course_country',)] = 'UK'

exploded_locations_filtered.head()

,provider_name,course_name,course_skills,course_locations,course_description,target_url,timestamp,course_country
0,Codez Academy,Digital Roots Scheme,CSS,Wales,"Students will learn three main languages: CSS,...",https://www.coursereport.com/schools/codez-aca...,2023-05-25 14:12:01.312105,UK
1,Codez Academy,Digital Roots Scheme,HTML,Wales,"Students will learn three main languages: CSS,...",https://www.coursereport.com/schools/codez-aca...,2023-05-25 14:12:01.312105,UK
2,Codez Academy,Digital Roots Scheme,JavaScript,Wales,"Students will learn three main languages: CSS,...",https://www.coursereport.com/schools/codez-aca...,2023-05-25 14:12:01.312105,UK
3,Codez Academy,Digital Roots Scheme,Javascript,Wales,"Students will learn three main languages: CSS,...",https://www.coursereport.com/schools/codez-aca...,2023-05-25 14:12:01.312105,UK
4,Codez Academy,Foundation Course in Front-end Development,JavaScript,Wales,Students will create two websites (a 1-page we...,https://www.coursereport.com/schools/codez-aca...,2023-05-25 14:12:01.312105,UK


### 11. Create dataframe of all unique skills

In [229]:
skills_df = normalised_courses['course_skills'].explode('course_skills').drop_duplicates().reset_index().drop('index', axis=1)

skills_df.head()

,course_skills
0,CSS
1,HTML
2,JavaScript
3,Javascript
4,Algorithms


### You can output to a csv by running the block below:

In [230]:
exploded_locations_filtered.to_csv("./processed_data.csv")
skills_df.to_csv("./processed_skills_data.csv")